In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import os

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [106]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
def visualize(tensor, title, path=".", save=False):

  detached_tensor = tensor.cpu().detach()
  
  image_grid = make_grid(detached_tensor, nrow=8)
  
  plt.imshow(image_grid.permute(1, 2, 0).squeeze())
  plt.title(title)
  
  if save:
    plt.savefig(os.path.join(path, title + ".png"))
  
  plt.show()

## Dataset

In [ ]:
!mkdir "/content/celebA"
!sudo unzip "/content/drive/MyDrive/img_align_celeba.zip" -d "/content/celebA"

In [64]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets import CIFAR10
from PIL import Image
from torch.utils.data import Subset

In [65]:
lr = 0.001
batch_size = 16
beta1 = 0
beta2 = 0.99
criterion = nn.BCELoss()
c_lambda = 10
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
# class CelebA(Dataset):
  
#   def __init__(self, root, transform=None):
#     self.files = glob.glob(os.path.join(root, "*.jpg"))
#     self.transform = transform

#   def __getitem__(self, index):

#     image = Image.open(self.files[index]) 

#     if self.transform is not None:
#       return self.transform(image)
    
#     return image

#   def __len__(self):
#     return len(self.files)

In [87]:
def loaderFunc(transform):
  
  # ds = CIFAR10('~/.torch/data/', train=True, download=True, transform=transform)
  # dog_indices, deer_indices, other_indices = [], [], []
  # dog_idx, deer_idx = ds.class_to_idx['dog'], ds.class_to_idx['airplane']

  # for i in range(len(ds)):
  #   current_class = ds[i][1]
  #   if current_class == dog_idx:
  #     dog_indices.append(i)
  #   elif current_class == deer_idx:
  #     deer_indices.append(i)
  #   else:
  #     other_indices.append(i)
  # dog_indices = dog_indices[:int(0.6 * len(dog_indices))]
  # deer_indices = deer_indices[:int(0.6 * len(deer_indices))]
  # new_dataset = Subset(ds, deer_indices)
  
  # dataset = CelebA("/content/drive/MyDrive/haircolors/images/img_align_celeba/", transform=transform)
  # dataset = CIFAR10('~/.torch/data/', train=True, download=True, transform=transform)
  dataset = ImageFolder("/content/celebA", transform=transform)

  train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
  )

  return train_loader

In [88]:
def sample_data(image_size=4):

  transform = transforms.Compose([
      transforms.Resize(image_size),
      transforms.CenterCrop(image_size),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])


  loader = loaderFunc(transform)

  for img, label in loader:
      yield (img, label)

<img src="https://machinelearningmastery.com/wp-content/uploads/2019/06/Tables-Showing-Generator-and-Discriminator-Configuration-for-the-Progressive-Growing-GAN.png"  width="1024"/>

In [68]:
class PixelNorm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        # assuming input is in the from of (batch_size, channels, width, height)
        return input / torch.sqrt(torch.mean(input ** 2, dim=1, keepdim=True)
                                  + 1e-8)

In [69]:
from math import sqrt 
def EqualLR(module, input):

  weight = getattr(module, 'weight')
  del module._parameters['weight']
  weight = weight * sqrt(2 / (weight.size(1) * weight[0][0].numel()))
  module.register_parameter("weight", nn.Parameter(weight.data))
  return input

In [70]:
class EqualConv2d(nn.Module):
  
  def __init__(self, *args, **kwargs):
    
    super().__init__()

    conv = nn.Conv2d(*args, **kwargs)
    conv.weight.data.normal_()
    conv.bias.data.zero_()
    conv.register_forward_pre_hook(EqualLR)
    self.conv = conv
    
  def forward(self, x):
    return self.conv(x)

In [71]:
def GeneratorBlock(in_channel, out_channel, output_size, first_block=False):
  # Growing gradually to 1024 * 1024 is done by incrementally adding blocks
  # in this function we get specification of the block and return it
  # for example input would be 4 * 4 and output_size would be 8 * 8
  if first_block:

    model = nn.Sequential(
      EqualConv2d(in_channel, out_channel, kernel_size=4, padding=3),
      nn.LeakyReLU(0.2),
      PixelNorm(),

      EqualConv2d(out_channel, out_channel, kernel_size=3, padding=1),
      nn.LeakyReLU(0.2),
      PixelNorm(),
    )
  else:

    model = nn.Sequential(
    
      nn.Upsample((output_size, output_size), mode='bilinear', align_corners=True),
      EqualConv2d(in_channel, out_channel, kernel_size=3, padding=1),
      nn.LeakyReLU(0.2),
      PixelNorm(),

      EqualConv2d(out_channel, out_channel, kernel_size=3, padding=1),
      nn.LeakyReLU(0.2),
      PixelNorm(),
    )

  return model  

In [72]:
class Generator(nn.Module):

  def __init__(self):
    
    super().__init__()

    self.block4_4 = GeneratorBlock(512, 512, 4, first_block=True)
    self.block8_8 = GeneratorBlock(512, 512, 8)
    self.block16_16 = GeneratorBlock(512, 512, 16)
    self.block32_32 = GeneratorBlock(512, 512, 32)
    self.block64_64 = GeneratorBlock(512, 256, 64)
    self.block128_128 = GeneratorBlock(256, 128, 128)

    self.blocks = nn.ModuleList([
        self.block4_4,
        self.block8_8,
        self.block16_16,
        self.block32_32,
        self.block64_64,
        self.block128_128
    ])

    self.to_rgbs = nn.ModuleList([
      nn.Conv2d(512, 3, 1),
      nn.Conv2d(512, 3, 1),
      nn.Conv2d(512, 3, 1),
      nn.Conv2d(512, 3, 1),
      nn.Conv2d(256, 3, 1),
      nn.Conv2d(128, 3, 1),
    ])


  def forward(self, x, step, alpha):
    # we have six steps toward progressively increasing the output size
    # alpha is the weight of output of new block compared to upsampled input
    if step == 1: # no need to average
      out = self.blocks[0](x)
      out = self.to_rgbs[0](out)

    elif step > 1:

      for block in self.blocks[:step - 1]: # assuming all previous blocks have been trained completely
        x = block(x)

      # x_small_block = self.blocks[step-2](x) # 256 * 64 * 64
      # x is 16 * 512 * 16 * 16
      x_large_block = self.blocks[step-1](x) # 128 * 128 * 128
      x_large_image = self.to_rgbs[step-1](x_large_block) # 3 * 128 * 128


      x_small_upsample = F.interpolate(x, x_large_image.shape[-2:]) # 3 * 64 * 64
      x_upsample_rgb = self.to_rgbs[step-2](x_small_upsample)

      out = (alpha *  x_large_image) + (1 - alpha) * (x_upsample_rgb)


    return out


In [73]:
gen = Generator().to(device)

In [74]:
gen(torch.randn(16, 512, 1, 1).to(device), 5, 0.2).shape

torch.Size([16, 3, 64, 64])

In [75]:
def DiscriminatorBlock(in_channel, out_channel, output_size, last_block=False):
  # Growing gradually to 1024 * 1024 is done by incrementally adding blocks
  # in this function we get specification of the block and return it
  # for example input would be 4 * 4 and output_size would be 8 * 8
  if last_block:

    model = nn.Sequential(
      nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1),
      nn.LeakyReLU(0.2),

      nn.Conv2d(out_channel, out_channel, kernel_size=4, padding=0),
      nn.LeakyReLU(0.2),
      nn.Flatten(start_dim=1),
      nn.Linear(512, 1),
    )
  else:

    model = nn.Sequential(
    
      EqualConv2d(in_channel, out_channel, kernel_size=3, padding=1),
      PixelNorm(),
      nn.LeakyReLU(0.2),

      EqualConv2d(out_channel, out_channel, kernel_size=3, padding=1),
      PixelNorm(),
      nn.LeakyReLU(0.2),

      nn.AvgPool2d(kernel_size=2),
    )

  return model

In [76]:
class Discriminator(nn.Module):

  def __init__(self):
    
    super().__init__()

    self.block64_64 = DiscriminatorBlock(128, 256, 64)
    self.block32_32 = DiscriminatorBlock(256, 512, 32)
    self.blcok16_16 = DiscriminatorBlock(512, 512, 16)
    self.block8_8 = DiscriminatorBlock(512, 512, 8)
    self.block4_4 = DiscriminatorBlock(512, 512, 4)
    self.block1_1 = DiscriminatorBlock(513, 512, 1, last_block=True)

    self.blocks = nn.ModuleList([
      self.block64_64,
      self.block32_32,
      self.blcok16_16,
      self.block8_8,
      self.block4_4,
      self.block1_1
    ])

    self.from_rgbs = nn.ModuleList([
      nn.Conv2d(3, 128, 1),
      nn.Conv2d(3, 256, 1),
      nn.Conv2d(3, 512, 1),
      nn.Conv2d(3, 512, 1),
      nn.Conv2d(3, 512, 1),
      nn.Conv2d(3, 512, 1),
    ])
  

  def forward(self, x_large, step, alpha):
      
    # alpha
    

    if step == 1:
      x_large = self.from_rgbs[(6-step)](x_large) # input : 3 * 8 * 8 and output is 512 * 8 * 8
      minibatch_std = x_large.std(0).mean() # 1
      minibatch_std = minibatch_std.expand(x_large.size(0), 1, 4, 4) # 16, 1, 4, 4
      x_large = torch.cat([x_large, minibatch_std], dim=1)
      out = self.blocks[-step](x_large) # last layer output is 512 * 4 * 4
      return out

    out = self.blocks[-step](self.from_rgbs[6 - step](x_large)) # last layer output is 512 * 4 * 4
    
    if step > 1:
    
      x_large_downsampled = self.from_rgbs[(6 - step) + 1](F.avg_pool2d(x_large, kernel_size=2)) # input 3 * 8 * 8 => 3 * 4 *4  and then from rgb to 512 * 4 *4 

      out = (1 - alpha) * x_large_downsampled + (alpha * out)


      # intermidiate lay
      for block in self.blocks[(6 - step) + 1: -1]:
        out = block(out)

      # last layer needs mini-batch variation
      minibatch_std = x_large.std(0).mean() # 1
      minibatch_std = minibatch_std.expand(x_large.size(0), 1, 4, 4) # 16, 1, 4, 4
      out = torch.cat([out, minibatch_std], dim=1)
      out = self.blocks[5](out)
    
    return out

In [77]:
disc = Discriminator().to(device)

In [78]:
disc(torch.randn(16, 3, 64, 64).to(device), 5, 0.2).shape

torch.Size([16, 1])

In [79]:
gen_optim = torch.optim.Adam(gen.parameters(), lr=lr , betas=(beta1, beta2))
disc_optim = torch.optim.Adam(disc.parameters(), lr=lr , betas=(beta1, beta2))

In [80]:
from tqdm.auto import tqdm

In [81]:
def gradient_penalty(real, fake, epsilon, step, alpha):
  
  mixed_images = epsilon * real + (1 - epsilon) * fake
  mixed_scores = disc(mixed_images, step, alpha)

  gradients = torch.autograd.grad(
      outputs=mixed_scores,
      inputs=mixed_images,
      grad_outputs=torch.ones_like(mixed_scores),
      retain_graph=True,
      create_graph=True,
  )[0]

  flat_gradients = gradients.reshape(len(gradients), -1)

  norm = torch.norm(flat_gradients, dim=1)

  gp = torch.mean((norm - 1) ** 2)

  return gp

In [109]:
# step and alpha
step = 1
step_size = 0.00002
alpha = 0
latent_space_size = 512
i = 0
stabilize = False

In [ ]:
ckpt = torch.load("/content/drive/MyDrive/progressive_gan_checkpoints/cpt_step_4_epoch_20_iteration_2000")
disc.load_state_dict(ckpt["disc_model"])
gen.load_state_dict(ckpt["gen_model"])
disc_optim.load_state_dict(ckpt["disc_optim"])
gen_optim.load_state_dict(ckpt["gen_optim"])
epoch_t = ckpt["epoch"]
alpha = ckpt["alpha"]
step = ckpt["step"]

In [83]:
def accumulate(model1, model2, weight=0.999):
  
  model1_params = dict(model1.named_parameters())
  model2_params = dict(model2.named_parameters())

  for key in model1_params.keys():
    model1_params[key].data.mul_(weight).data.add_((1 - weight) * model2_params[key].data)


In [60]:
### Unit Test 

# gen1 = Generator()
# gen2 = Generator()
# accumulate(gen1, gen2, 0.5)

In [84]:
g_running = Generator()
g_running.train(False)
accumulate(g_running, gen, 0)

In [85]:
import numpy as np
disc_loss_list = []
gen_loss_list = []

In [ ]:
pbar = tqdm(range(0, 600000))
dataset = sample_data(2 ** (step + 1))

for iteration in pbar:

  alpha = min(1, i * step_size)

  if stabilize == False and i > 50000:
    stabilize = True
    alpha = 1
    dataset = sample_data(2 ** (step + 1))
  
  if i > 100000:
    alpha = 0
    step += 1
    stabilize = False
    i = 0
    
    if step > 6:
      step = 6
      alpha = 0

    dataset = sample_data(2 ** (step + 1))
  i += 1

  
  
  
  real, _ = next(dataset)

  real = real.to(device)

  # Training Discriminator
  disc_optim.zero_grad()

  fake = gen(torch.randn(len(real), latent_space_size, 1, 1).to(device), step, alpha).detach()
  pred_fake = disc(fake, step, alpha)

  pred_real = disc(real, step, alpha)

  epsilon = torch.randn(len(real), 1, 1, 1, requires_grad=True).to(device)

  gp = gradient_penalty(real, fake, epsilon, step, alpha)

  disc_loss = - torch.mean(pred_real - pred_fake) + (c_lambda * gp)

  disc_loss.backward()

  disc_optim.step()

  # Training Generator
  gen_optim.zero_grad()

  fake = gen(torch.randn(batch_size, latent_space_size, 1, 1).to(device), step, alpha)
  pred_fake = disc(fake, step, alpha).reshape(-1)
  gen_loss = - torch.mean(pred_fake)

  gen_loss.backward()

  gen_optim.step()

  accumulate(g_running, gen)


  disc_loss_list.append(disc_loss.item())
  gen_loss_list.append(gen_loss.item())

  pbar.set_description(
    "Generator loss : {:.5f}, Discriminator loss: {:.5f}, alpha is {:.5f} and step is {}".format(gen_loss.item(), disc_loss.item(), alpha, step)
  )

  if iteration % 2500 == 0:
    
    fake_samples = g_running(
        torch.randn(16, latent_space_size, 1, 1),
        step, 
        alpha,
    )
    # visualize(fake_samples, "linear_{}".format(iteration))
    visualize((torch.tanh(fake_samples) + 1) / 2, "tanh_{}".format(iteration), path="/content/samples", save=True)
    visualize(real, "Real")
    
  
  if iteration % 5000 == 0:
    torch.save({
        "gen_model": gen.state_dict(),
        "gen_optim": gen_optim.state_dict(),
        "disc_model": disc.state_dict(),
        "disc_optim": disc_optim.state_dict(),
        "iteration": iteration,
        "step": step,
        "alpha": alpha,
    }, "/content/drive/MyDrive/progressive_gan_checkpoints/cpt_step_{}_epoch_{}_iteration_{}".format(step, epoch, iteration))

    